# NN - Churn Modelling

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [48]:
# import data
dataset = pd.read_csv("Churn_Modelling.csv")
dataset.shape

(10000, 14)

In [33]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [38]:
X = dataset.iloc[:, 3:13].values
y= dataset.iloc[:,13].values

In [39]:
# categorical variables

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:,1] = labelencoder_X_1.fit_transform(X[:,1])

labelencoder_X_2 = LabelEncoder()
labelencoder_X_1 = LabelEncoder()
X[:,2] = labelencoder_X_2.fit_transform(X[:,2])

onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()

# remove the reference column
X = X[: , 1:]

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=0)

In [44]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [77]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [78]:
# initializing the ANN
classifier = Sequential()

# adding the input layer and the first hidden layer
classifier.add(Dense(6 ,   activation='relu',  kernel_initializer='random_uniform', input_shape = (X_train.shape[1],)))
classifier.add(Dense(6 ,   activation='relu',  kernel_initializer='random_uniform'))

# add the output layer
classifier.add(Dense(1 ,   activation='sigmoid',  kernel_initializer='random_uniform'))

# compile the ANN
classifier.compile(optimizer = 'adam',  loss ='binary_crossentropy', metrics = ['accuracy'] )

In [79]:
classifier.fit(X_train, y_train, batch_size = 10, epochs =20)

Epoch 1/20
8000/8000 [==============================] - 4s 551us/step - loss: 0.4864 - acc: 0.7949
Epoch 2/20
8000/8000 [==============================] - 2s 208us/step - loss: 0.4280 - acc: 0.7960
Epoch 3/20
8000/8000 [==============================] - 2s 203us/step - loss: 0.4236 - acc: 0.7960
Epoch 4/20
8000/8000 [==============================] - 2s 191us/step - loss: 0.4190 - acc: 0.8181
Epoch 5/20
8000/8000 [==============================] - 3s 341us/step - loss: 0.4167 - acc: 0.8269
Epoch 6/20
8000/8000 [==============================] - 7s 921us/step - loss: 0.4148 - acc: 0.8297
Epoch 7/20
8000/8000 [==============================] - 6s 757us/step - loss: 0.4131 - acc: 0.8324
Epoch 8/20
8000/8000 [==============================] - 6s 727us/step - loss: 0.4123 - acc: 0.8312
Epoch 9/20
8000/8000 [==============================] - 5s 605us/step - loss: 0.4111 - acc: 0.8332
Epoch 10/20
8000/8000 [==============================] - 7s 881us/step - loss: 0.4104 - acc: 0.8325
Epoch 11/

In [82]:
y_pred = classifier.predict_classes(X_test)


# y_pred = classifier.predict(X_test)
# y_pred = (y_pred>0.5)

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

In [83]:
cm

array([[1552,   43],
       [ 280,  125]])

In [97]:
# accuracy
sum([p[0] for p in y_pred]==y_test)/len(y_test)

0.83850000000000002